## 1. 使用分组值填充缺失值
- 对于有缺失值的数据，可以使用`fillna()`使用指定的数据去填充缺失值
- 对于分组后的各组数据里的缺失值，如果需要使用与所属分组有关的数据来填充缺失值，可以使用`apply()`调用一个填充缺失值功能的函数来处理
- 也可以使用指定的值去填充不同分组的缺失值

In [1]:
import numpy as np
import pandas as pd

In [2]:
s=pd.Series(np.random.randn(10))
s[[2,5,8,9]]=np.nan
s

0   -1.038213
1    1.489871
2         NaN
3    1.298391
4   -1.187405
5         NaN
6    0.309498
7    0.067049
8         NaN
9         NaN
dtype: float64

In [3]:
# 使用平均值填充所有缺失值
s.fillna(s.mean())

0   -1.038213
1    1.489871
2    0.156532
3    1.298391
4   -1.187405
5    0.156532
6    0.309498
7    0.067049
8    0.156532
9    0.156532
dtype: float64

In [4]:
# 对数据根据索引进行奇偶分组
f=lambda x:'偶数' if x%2==0 else '奇数'
s.groupby(f).agg(['mean','count','sum'])

,mean,count,sum
偶数,-0.638707,3,-1.916120
奇数,0.951770,3,2.855311


In [5]:
# fillna_s：使用传入的数据的平均值来填充数据的缺失值
def fillna_s(x):
    return x.fillna(x.mean())

# 使用apply，将groupby后的每个分组分别传给fillna_s函数
# 即使用每个分组的平均值来填充该分组的缺失值
s.groupby(f).apply(fillna_s)

0   -1.038213
1    1.489871
2   -0.638707
3    1.298391
4   -1.187405
5    0.951770
6    0.309498
7    0.067049
8   -0.638707
9    0.951770
dtype: float64

In [6]:
# 为每个分组指定不同的缺失值
fill_value={'奇数':99,'偶数':88}
fna=lambda x:x.fillna(fill_value[x.name])
s.groupby(f).apply(fna)

0    -1.038213
1     1.489871
2    88.000000
3     1.298391
4    -1.187405
5    99.000000
6     0.309498
7     0.067049
8    88.000000
9    99.000000
dtype: float64

## 2.随机采样
目的：建立一副52张牌的扑克，每个花色随机抽取5张

In [26]:
card_suits=list('桃杏梅方')
card_letter={1:'A',11:'J',12:'Q',13:'K'}
# 建立A~K共13个字符格式的牌面值组成的列表
card_num=[card_letter[x] if x==1 or x>10 else str(x) for x in np.arange(1,14)]
cards=[]
# 建立4种花色共52张牌的列表
for suit in card_suits:
    cards.extend([num+suit for num in card_num])
cards=pd.Series(card_num*4,index=cards)
cards[:13]

A桃      A
2桃      2
3桃      3
4桃      4
5桃      5
6桃      6
7桃      7
8桃      8
9桃      9
10桃    10
J桃      J
Q桃      Q
K桃      K
dtype: object

In [46]:
# 随机获取5张牌
def draw(cards,n=5):
    return cards.sample(n)
draw(cards)

5杏    5
9杏    9
8方    8
7杏    7
7方    7
dtype: object

In [39]:
# groupby()里的函数是对索引进行处理的
# 根据索引的最后字符即花色进行分组，将每个分组代入draw()中随机抽取5张牌
cards.groupby(lambda x:x[-1]).apply(draw,5)

方  A方      A
   10方    10
   Q方      Q
   3方      3
   K方      K
杏  4杏      4
   A杏      A
   9杏      9
   Q杏      Q
   K杏      K
桃  Q桃      Q
   K桃      K
   8桃      8
   2桃      2
   J桃      J
梅  A梅      A
   5梅      5
   3梅      3
   9梅      9
   7梅      7
dtype: object

In [58]:
used_cards=draw(cards,13).index
used_cards

Index(['4桃', '4杏', '3方', '8桃', '10桃', 'K梅', '9桃', '5方', '5梅', 'Q杏', '4梅', 'J杏',
       '2梅'],
      dtype='object')

In [56]:
cards.index

Index(['A桃', '2桃', '3桃', '4桃', '5桃', '6桃', '7桃', '8桃', '9桃', '10桃', 'J桃', 'Q桃',
       'K桃', 'A杏', '2杏', '3杏', '4杏', '5杏', '6杏', '7杏', '8杏', '9杏', '10杏', 'J杏',
       'Q杏', 'K杏', 'A梅', '2梅', '3梅', '4梅', '5梅', '6梅', '7梅', '8梅', '9梅', '10梅',
       'J梅', 'Q梅', 'K梅', 'A方', '2方', '3方', '4方', '5方', '6方', '7方', '8方', '9方',
       '10方', 'J方', 'Q方', 'K方'],
      dtype='object')

In [77]:
cards[~cards.index.isin(used_cards)]

A桃      A
2桃      2
3桃      3
5桃      5
6桃      6
7桃      7
J桃      J
Q桃      Q
K桃      K
A杏      A
2杏      2
3杏      3
5杏      5
6杏      6
7杏      7
8杏      8
9杏      9
10杏    10
K杏      K
A梅      A
3梅      3
6梅      6
7梅      7
8梅      8
9梅      9
10梅    10
J梅      J
Q梅      Q
A方      A
2方      2
4方      4
6方      6
7方      7
8方      8
9方      9
10方    10
J方      J
Q方      Q
K方      K
dtype: object